In [1]:
# This notebook contains code that classifies a book title into a particular genre - fiction or non-fiction

In [1]:
# import libraries
import pandas as pd
import numpy as np
from transformers import pipeline

/Users/eugene/Personal_Projects/ML_Project/nlp_projects/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
df =  pd.read_csv('./books_cleaned.csv')

In [3]:
df.columns

Index(['isbn13', 'isbn10', 'title', 'authors', 'categories', 'thumbnail',
       'description', 'published_year', 'average_rating', 'num_pages',
       'ratings_count', 'title_subtitle', 'tagged_desc'],
      dtype='object')

In [5]:
df['categories'].value_counts().reset_index()

categories  count
0                      Fiction   2111
1             Juvenile Fiction    390
2    Biography & Autobiography    311
3                      History    207
4           Literary Criticism    124
..                         ...    ...
474               Conspiracies      1
475       Brothers and sisters      1
476             Rock musicians      1
477             Community life      1
478    Indic fiction (English)      1

[479 rows x 2 columns]

In [8]:
# isolate categories that have at least 50 books or more
df['categories'].value_counts().reset_index().query('count > 50')


categories  count
0                     Fiction   2111
1            Juvenile Fiction    390
2   Biography & Autobiography    311
3                     History    207
4          Literary Criticism    124
5                  Philosophy    117
6                    Religion    117
7     Comics & Graphic Novels    116
8                       Drama     86
9         Juvenile Nonfiction     57
10                    Science     56
11                     Poetry     51

In [9]:
# let's look at juvenile fiction category
df[df['categories']=='Juvenile Fiction']

isbn13      isbn10  \
30    9780006646006  000664600X   
79    9780020442608  0020442602   
85    9780030547744  0030547741   
86    9780060000141  0060000147   
87    9780060001537  0060001534   
...             ...         ...   
4845  9781590385814  1590385810   
4930  9781596792500  1596792507   
4942  9781599900056  159990005X   
5010  9781844580514  1844580512   
5157  9781932416374  1932416374   

                                                  title  \
30                                   Ocean Star Express   
79                       The voyage of the Dawn Treader   
85                             Where the Red Fern Grows   
86                                       Poppy's Return   
87                                    Diary of a Spider   
...                                                 ...   
4845                                         Fablehaven   
4930  Sherlock Holmes and the Case of the Hound of t...   
4942                                    The Drift House   
5010                               Attack of the Jaguar   
5157                The Very Persistent Gappers of Frip   

                                  authors        categories  \
30               Mark Haddon;Peter Sutton  Juvenile Fiction   
79                    Clive Staples Lewis  Juvenile Fiction   
85                           Wilson Rawls  Juvenile Fiction   
86                                    Avi  Juvenile Fiction   
87                          Doreen Cronin  Juvenile Fiction   
...                                   ...               ...   
4845                         Brandon Mull  Juvenile Fiction   
4930  Arthur Conan Doyle;Malvina G. Vogel  Juvenile Fiction   
4942                            Dale Peck  Juvenile Fiction   
5010                         M. A. Harvey  Juvenile Fiction   
5157           George Saunders;Lane Smith  Juvenile Fiction   

                                              thumbnail  \
30    http://books.google.com/books/content?id=I2QZA...   
79    http://books.google.com/books/content?id=fDD3C...   
85    http://books.google.com/books/content?id=IHpRw...   
86    http://books.google.com/books/content?id=XbcMJ...   
87    http://books.google.com/books/content?id=UWvZo...   
...                                                 ...   
4845  http://books.google.com/books/content?id=tbVIP...   
4930  http://books.google.com/books/content?id=EWgWP...   
4942  http://books.google.com/books/content?id=kbwPY...   
5010  http://books.google.com/books/content?id=3HUdt...   
5157  http://books.google.com/books/content?id=xYJOP...   

                                            description  published_year  \
30    Joe and his parents are enjoying a summer holi...          2002.0   
79    The "Dawn Treader" is the first ship Narnia ha...          1970.0   
85    A young boy living in the Ozarks achieves his ...          2000.0   
86    There's trouble at Gray House, the girlhood ho...          2006.0   
87    This is the diary ... of a spider. But don't b...          2005.0   
...                                                 ...             ...   
4845  When Kendra and Seth go to stay at their grand...          2006.0   
4930  Sherlock Holmes and Dr. Watson travel to the b...          2005.0   
4942  Sent to stay with their uncle in a ship-like h...          2006.0   
5010  This training manual for operatives of Xtreme ...          2004.0   
5157  A novel set in the seaside village of Frip fol...          2006.0   

      average_rating  num_pages  ratings_count  \
30              3.50       32.0            1.0   
79              4.09      216.0         2869.0   
85              4.37      288.0           95.0   
86              3.99      256.0         1086.0   
87              4.25       40.0         7903.0   
...              ...        ...            ...   
4845            4.09      351.0       111896.0   
4930            4.51      237.0           28.0   
4942            3.64      437.0          595.0   
5010            

In [ ]:
# lets transform our categories into two major categories only
category_mapping = {
    'Fiction' :  'Fiction',
    'Juvenile Fiction': 'Children Fiction', 
    'Biography & Autobiography' : '',
    'History', 
    'Literary Criticism', 
    'Philosophy', 
    'Religion',
    'Comics & Graphic Novels', 
    'Drama', 
    'Juvenile Nonfiction',
    'Science', 
    'Poetry'
    
}